In [2]:
# Ignore warnings.
import warnings
warnings.filterwarnings("ignore")

# obtain reproducible results
import numpy as np
import tensorflow as tf
import random as rn
from keras import backend as K

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(42)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
rn.seed(12345)
# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
# https://www.tensorflow.org/api_docs/python/tf/set_random_seed
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# Rest of code follows ...

# Check GPU device.
print(K.tensorflow_backend._get_available_gpus())

['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
#https://richliao.github.io/supervised/classification/2016/11/26/textclassifier-convolutional/
#https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
#RNN

import os
import pandas as pd
import re
from tqdm import tqdm
from multiprocessing import Pool
from spellchecker import SpellChecker
import string
import math

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
import matplotlib.pyplot as plt

# For encoding labels.
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [4]:
df_coded_tweet=pd.read_csv('../../dataset/muy060_suppl_supplementary_appendix/muy060_Appendix_2.csv')
df_coded_tweet.loc[df_coded_tweet.JasonCode==2, 'moral_bin']=1
df_coded_tweet.moral_bin.fillna('0', inplace=True)
df_coded_tweet['moral_bin']=[int(s) for s in df_coded_tweet.moral_bin]
df_coded_tweet.sample(5)

,HITID,Text,Answer1,Answer2,Agreement,Answer,Date,JasonCode,moral_bin
65,3AXFSPQOYQYNNLFNVXHLO2FHNRHFJ3,RT NWSNewOrleans Our 6ft 6in MIC next to 11ft ...,None of the above.,Performative Reputation,No,no_agreement,2018-07-06 18:03:42 UTC,0,0
114,3N5YJ55YXG3OAKP0ZFNL38L7ZV9ANP,RT fema Storm surge can be one of the most dan...,None of the above.,Technical Reputation,No,no_agreement,2018-07-06 17:59:34 UTC,2,1
16,35A1YQPVFEGZQD2S73JCQP94LIEI5B,RT statedeptspox Thank you Dr Alison Mann fro...,None of the above.,None of the above.,Yes,None of the above.,2018-07-06 17:59:48 UTC,0,0
141,3TX9T2ZCB91FYM6M38U7GKP71SJWZ2,CommerceGov announces preliminary countervaili...,None of the above.,Performative Reputation,No,no_agreement,2018-07-06 18:04:45 UTC,1,0
156,3GVPRXWRPHUEDHGBVWY9O9N0CO97IF,Whos a good boy?A USAirForce airman rewards Ni...,Moral Reputation,Technical Reputation,No,no_agreement,2018-07-06 18:08:55 UTC,2,1


In [5]:
# Prepare labels
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

lb.fit(df_coded_tweet.moral_bin.unique())
y_train=lb.transform(df_coded_tweet['moral_bin'])
from keras.utils import to_categorical
y_train = to_categorical(y_train)

# lb.fit(df_coded_tweet.JasonCode.unique())
# y_train=lb.transform(df_coded_tweet['JasonCode'])

# y_test=lb.transform(df_test['broad_cat']) # No need to transform Y for testing.

In [6]:
text_token_list_train=df_coded_tweet['Text']

In [7]:
# Build word index for train and validation texts.
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_token_list_train.to_list())
print(list(tokenizer.word_index.items())[0:5])

[('the', 1), ('to', 2), ('rt', 3), ('of', 4), ('a', 5)]


In [8]:
# Text to sequences.
seq_encoding_text_train=tokenizer.texts_to_sequences(text_token_list_train)

# Pads sequences to the same length (i.e., prepare matrix).
x_train=pad_sequences(sequences=seq_encoding_text_train,
                      maxlen=max([len(s) for s in seq_encoding_text_train]), # Max length of the sequence.
                      dtype = "int32", padding = "post", truncating = "post", 
                      value = 0 # Zero is used for representing None or Unknown.
                     )

In [9]:
import gensim.downloader as api
EMBEDDING_DIM=100
glove_word_vector=api.load('glove-wiki-gigaword-'+str(EMBEDDING_DIM))

In [10]:
embedding_matrix = np.zeros((len(tokenizer.word_index)+1, EMBEDDING_DIM)) # Plus one: embedding matrix starts from 0, word index starts from 1.

In [11]:
for word, index in tokenizer.word_index.items():
    try:
        embedding_matrix[index] = glove_word_vector.get_vector(word)
    except:
        pass
        # words not found in embedding index will be all-zeros.

In [12]:
# https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, # Size of vocabulary.
                            input_length=max([len(s) for s in seq_encoding_text_train]), # Length of input, i.e., length of padded sequence.
                            output_dim=EMBEDDING_DIM, # Size of the vector space in which words will be embedded.
                            weights=[embedding_matrix],
                            trainable=False)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalMaxPool1D, Conv1D
from datetime import datetime
import itertools

num_filters=[32, 64, 128]
kernel_size=[3,5,7]
conv_act=['sigmoid', 'softplus', 'tanh', 'softmax']
act_32=['sigmoid', 'softplus', 'tanh', 'softmax']
act_16=['sigmoid', 'softplus', 'tanh', 'softmax']
out_act=['sigmoid', 'softplus', 'tanh', 'softmax']
param_list=list(itertools.product(num_filters, kernel_size, conv_act, act_32, act_16, out_act))

In [ ]:
result_list=[]

for num_filters, kernel_size, conv_act, act_32, act_16, out_act in tqdm(param_list):
    with tf.device('/gpu:0'): # Small dataset, may even increase time because of data transfer between C/GPU.
        model = Sequential()
        model.add(embedding_layer)
        # model.add(Flatten())
        model.add(Conv1D(num_filters, kernel_size, activation=conv_act))
        model.add(GlobalMaxPool1D())
        model.add(Dense(units=32, activation=act_32))
        model.add(Dense(units=16, activation=act_16))
        model.add(Dense(units=len(y_train[0]), activation=out_act))
        # compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
        # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
        # fit the model
        history=model.fit(x_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    result_list+=[[max(history.history['val_acc']), [num_filters, kernel_size, conv_act, act_32, act_16, out_act]]]
    ############### Clean up and rebuid objects; otherwise, tensor graph becomes larger and larger and requires more time. ##########
    del model
    K.clear_session() # Will cause error: Tensor must from the same graph. Cause: Embedding layers changed. Solution: rebuild embedding layer.
    ##### Initialize session for reproducibility #####
    np.random.seed(42)
    rn.seed(12345)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                                  inter_op_parallelism_threads=1)
    tf.set_random_seed(1234)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    ##################################################
    # Rebuid embedding layer.
    embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, # Size of vocabulary.
                                input_length=max([len(s) for s in seq_encoding_text_train]), # Length of input, i.e., length of padded sequence.
                                output_dim=EMBEDDING_DIM, # Size of the vector space in which words will be embedded.
                                weights=[embedding_matrix],
                                trainable=False)
    #################################################################################################################################

  0%|          | 0/2304 [00:00<?, ?it/s]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  3%|▎         | 78/2304 [03:38<1:42:44,  2.77s/it]

In [ ]:
pd.DataFrame(result_list).sort_values(0, ascending=False)[0:10]

### Train final model

In [22]:
############### Clean up and rebuid objects; otherwise, tensor graph becomes larger and larger and requires more time. ##########
K.clear_session() # Will cause error: Tensor must from the same graph. Cause: Embedding layers changed. Solution: rebuild embedding layer.
##### Initialize session for reproducibility #####
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
##################################################
# Rebuid embedding layer.
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, # Size of vocabulary.
                            input_length=max([len(s) for s in seq_encoding_text_train]), # Length of input, i.e., length of padded sequence.
                            output_dim=EMBEDDING_DIM, # Size of the vector space in which words will be embedded.
                            weights=[embedding_matrix],
                            trainable=False)
#################################################################################################################################

model = Sequential()
model.add(embedding_layer)
# model.add(Flatten())
model.add(Conv1D(128, 5, activation='softmax'))
model.add(GlobalMaxPool1D())
model.add(Dense(units=32, activation='tanh'))
model.add(Dense(units=16, activation='sigmoid'))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
# fit the model
history=model.fit(x_train, y_train, validation_split=0.3, epochs=100, verbose=1)

Train on 138 samples, validate on 60 samples
Epoch 1/100
138/138 [==============================] - 0s 3ms/step - loss: 0.6789 - acc: 0.5870 - val_loss: 0.6791 - val_acc: 0.5833
Epoch 2/100
138/138 [==============================] - 0s 179us/step - loss: 0.6767 - acc: 0.5870 - val_loss: 0.6789 - val_acc: 0.5833
Epoch 3/100
138/138 [==============================] - 0s 185us/step - loss: 0.6735 - acc: 0.5870 - val_loss: 0.6785 - val_acc: 0.5833
Epoch 4/100
138/138 [==============================] - 0s 176us/step - loss: 0.6712 - acc: 0.5870 - val_loss: 0.6781 - val_acc: 0.5833
Epoch 5/100
138/138 [==============================] - 0s 177us/step - loss: 0.6676 - acc: 0.5870 - val_loss: 0.6778 - val_acc: 0.5833
Epoch 6/100
138/138 [==============================] - 0s 176us/step - loss: 0.6629 - acc: 0.5870 - val_loss: 0.6773 - val_acc: 0.5833
Epoch 7/100
138/138 [==============================] - 0s 180us/step - loss: 0.6564 - acc: 0.5870 - val_loss: 0.6757 - val_acc: 0.5833
Epoch 8/100


## Traps.

- Tried GridSearchCV, suffers from the "tensor graph becomes larger and larger" problem.

<s>
    
```Python
def create_model(num_filters, kernel_size, conv_act, act_first, act_second, out_act):
    model = Sequential()
    model.add(embedding_layer)
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation=conv_act))
    model.add(GlobalMaxPool1D())
    model.add(Dense(units=32, activation=act_first))
    model.add(Dense(units=16, activation=act_second))
    model.add(Dense(units=len(y_train[0]), activation=out_act))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
neural_network = KerasClassifier(build_fn=create_model, validation_split=0.3, verbose=0)
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)
grid_result = grid.fit(x_train, y_train, verbose=1)
```
    
</s>

- Give up trying multiprocessing: fail to send embedding layer (a pickle object) to threads.
    - possible solution: buid embedding layer on remote engines/threads.
        - Tried, looks like not working. Since it now only takes about 1.5 hours to complete the job, give up, move on.

<s>

```Python
import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:]

dview['Sequential']=Sequential
dview['embedding_layer']=embedding_layer
dview['Conv1D']=Conv1D
dview['GlobalMaxPool1D']=GlobalMaxPool1D
dview['Dense']=Dense

@dview.parallel(block=True)
def grid_search(params):
    model = Sequential()
    model.add(embedding_layer)
    # model.add(Flatten())
    model.add(Conv1D(params[0], params[1], activation=params[2]))
    model.add(GlobalMaxPool1D())
    model.add(Dense(units=32, activation=params[3]))
    model.add(Dense(units=16, activation=params[4]))
    model.add(Dense(units=len(y_train[0]), activation=params[5]))
    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
    # fit the model
    history=model.fit(x_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    return [[max(history.history['val_acc']), [num_filters, kernel_size, conv_act, act_32, act_16, out_act]]]

grid_search.map(param_list)

def grid_search(params):
    with tf.device('/gpu:0'): # Specify which GPU to use.
        model = Sequential()
        model.add(embedding_layer)
        # model.add(Flatten())
        model.add(Conv1D(params[0], params[1], activation=params[2]))
        model.add(GlobalMaxPool1D())
        model.add(Dense(units=32, activation=params[3]))
        model.add(Dense(units=16, activation=params[4]))
        model.add(Dense(units=len(y_train[0]), activation=params[5]))
        # compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
        # F1, precision, and recall removed. https://github.com/keras-team/keras/issues/5794
        # fit the model
        history=model.fit(x_train, y_train, validation_split=0.3, epochs=100, verbose=0)
    return [[max(history.history['val_acc']), [num_filters, kernel_size, conv_act, act_32, act_16, out_act]]]

from multiprocessing import Pool
# if __name__== "__main__":
p=Pool(10)
result_list=p.map(grid_search, param_list)
```

</s>